In [69]:
from random import randint, uniform
from time import time
from math import *
from numpy import arange
from scipy.optimize import curve_fit
from matplotlib import pyplot

In [158]:
def objective(z, a, b, c, d):
	return a * z + b * z**2 + c*z**3 + d

def plotter(x,y,k):
    # curve fit
    popt, _ = curve_fit(objective, x, y)
    # summarize the parameter values
    a, b, c = popt
    print('y = %.5f * x + %.5f * x^2 + %.5f * x^3 + %.5f' % (a, b, c, d))
    # plot input vs output
    pyplot.scatter(x, y)
    # define a sequence of inputs between the smallest and largest known inputs
    x_line = arange(min(x), max(x)+1, 1)
    # calculate the output for the range
    y_line = objective(x_line, a, b, c, d)
    # create a line plot for the mapping function
    pyplot.plot(x_line, y_line,'-', color=k)
    pyplot.show()

In [119]:
def absorandom2d(low, high,length):
    return [randint(low,high) for i in range(length)]

def absorandom3d(low,high,length,breadth):
    return [[randint(low,high) for i in range(length)] for j in range(breadth)]

def gen_row(breadth,arr, variance):
    for i in range(1,breadth):
        arr.append(arr[i-1]+randint(-variance,variance))
    return arr

def lessrandom3d(length, breadth):
    arr=gen_row(breadth,[0],4)
    for i in range(breadth):                        #mostly suited to smaller arrays... 500x500?
        arr[i]=gen_row(length,[arr[i]],1)
    return arr


def lessrandom3d_ver2(length,breadth, variance):
    arr=[gen_row(breadth,[0],variance)]
    for i in range(1,length):
        arr2=[]
        for j in range(breadth):
            arr2.append(arr[i-1][j]+randint(-variance*5//8,variance))
            if (arr2[j]-arr2[j-1]>variance) or (arr2[j-1]-arr2[j]>variance):
                arr2[j]=(arr2[j-1]+arr[i-1][j])//2
        arr.append(arr2)
    return arr

def pretty_print_1(arr):
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            print("{:>3}".format(arr[i][j]), end='')
        print()

def pretty_print_2(arr):
    for i in range(1000):
        for j in range(132):
            if arr[i][j]<-15:
                print(" ", end='')
            elif arr[i][j]<-5:
                print("-", end='')
            elif arr[i][j]<5:
                print("=", end='')
            elif arr[i][j]<15:
                print("&", end='')
            else:
                print("@", end='')
        print()

In [120]:
def hill_climbing_1D(arr,i):
    while True:
        current = arr[i]
        neighbor = [arr[max(0,i-1)],arr[min(len(arr)-1,i+1)]]
        indices = [max(0,i-1),min(len(arr)-1,i+1)]
        if max(neighbor) <= current:
            return (current,i)
        i = indices[neighbor.index(max(neighbor))]

In [121]:
def hill_climbing_2D(arr,i,j):
    m = len(arr)
    n = len(arr[0])
    while True:
        current = arr[i][j]
        neighbor = [arr[max(0,i-1)][j],arr[min(m-1,i+1)][j],arr[i][max(0,j-1)],arr[i][min(n-1,j+1)],arr[max(0,i-1)][max(0,j-1)],arr[max(0,i-1)][min(n-1,j+1)],arr[min(m-1,i+1)][max(0,j-1)],arr[min(m-1,i+1)][min(n-1,j+1)]]
        indices = [[max(0,i-1),j],[min(m-1,i+1),j],[i,max(0,j-1)],[i,min(n-1,j+1)],[max(0,i-1),max(0,j-1)],[max(0,i-1),min(n-1,j+1)],[min(m-1,i+1),max(0,j-1)],[min(m-1,i+1),min(n-1,j+1)]]
        if max(neighbor) <= current:
            return (current,i,j)
        i = indices[neighbor.index(max(neighbor))][0]
        j = indices[neighbor.index(max(neighbor))][1]

In [122]:
def simulated_annealing_1D(arr,i,time_limit,T):
    current = arr[i]
    m = len(arr)
    
    start_time = time()
    while (time() - start_time < time_limit):
        t = time_limit -time() + start_time
        x = randint(0,1)
        indices = [min(m-1,i+1),max(i-1,0)]
        neighbour = arr[indices[x]]
        E = neighbour - current
        #print(neighbour,indices[x])
        if E>=0:
            current = neighbour
            i = indices[x]
        elif t > 0 :
            z = uniform(0,1)
            if  z <= exp(E/T(t)):
                current = neighbour
                i= indices[x]
    return [current,time_limit,max(arr)-current]

In [123]:
def simulated_annealing_2D(arr,i,j, time_limit, T):
    current = arr[i][j]
    m = len(arr)
    n = len(arr[0])
    
    ilist = []
    for x in arr:
        ilist.append(max(x))
    
    start_time = time()
    while (time() - start_time < time_limit):
        t = time_limit - time() + start_time
        x = randint(0,7)
        indices = [[max(0,i-1),j],[min(m-1,i+1),j],[i,max(0,j-1)],[i,min(n-1,j+1)],[max(0,i-1),max(0,j-1)],[max(0,i-1),min(n-1,j+1)],[min(m-1,i+1),max(0,j-1)],[min(m-1,i+1),min(n-1,j+1)]]
        neighbour = arr[indices[x][0]][indices[x][1]]
        E = neighbour - current
        #print(neighbour,indices[x])
        if E>=0:
            current = neighbour
            [i,j]= indices[x]
        else :
            z = uniform(0,1)
            if  t!=0 and z <= exp(E/t) :
                current = neighbour
                [i,j]= indices[x]
    
    return [time_limit,max(ilist)-current]

In [188]:
def data2D_prob(sz):
    tl = 0 
    lit=[]
    for _ in range(25):
        tl+=0.2
        cnt = 0
        for lol in range(25):
            try:
                [x,y] = simulated_annealing_2D(lessrandom3d_ver2(sz,sz,2),randint(0,sz-1),randint(0,sz-1),tl,1)
            except OverflowError:
                ans = float('inf')
                
            if y==0:
                cnt = cnt + 1
        lit.append([tl,cnt*0.04])
        
    x = []
    k = []
    for _ in range(20):
        x.append(lit[_][0])
        k.append(lit[_][1])
    plotter(x,k,"red")

In [ ]:
data2D_prob(200)

In [ ]:
def data2D_err(sz):
    o = 1
    lit=[]
    for _ in range(10):
        o = o+0.2
        cnt = 0
        for lol in range(5):
            x,y = simulated_annealing_2D(lessrandom3d_ver2(sz,sz,1),sz//2,sz//2,o,1)
            if y!=0:
                cnt = cnt + 1
        lit.append([o,cnt*20])
    x = []
    y = []
    for _ in range(10):
        x.append(lit[_][0])
        y.append(lit[_][1])
    plotter(x,y,"red")

In [181]:
print(x)

2
